# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import re
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report,fbeta_score,make_scorer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
import pickle
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterDatabase.db')
df = pd.read_sql_table('Disasters', con = engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens 
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([("vect",CountVectorizer(tokenizer=tokenize)),
                    ("tfidf",TfidfTransformer()),
                    ("clf",MultiOutputClassifier(RandomForestClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 45)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
prediction = pipeline.predict(X_test)
print(classification_report(y_test, prediction, target_names = Y.columns))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      6034
               request       0.80      0.37      0.51      1306
                 offer       0.00      0.00      0.00        43
           aid_related       0.75      0.53      0.62      3249
          medical_help       0.58      0.08      0.14       605
      medical_products       0.68      0.10      0.18       378
     search_and_rescue       0.40      0.02      0.05       250
              security       0.67      0.01      0.03       151
              military       0.57      0.05      0.09       264
                 water       0.89      0.19      0.31       500
                  food       0.82      0.38      0.52       887
               shelter       0.83      0.19      0.31       656
              clothing       0.80      0.08      0.14       102
                 money       0.50      0.03      0.06       186
        missing_people       0.33      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd17647b1e0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [9]:
parameters = { 'clf__estimator__n_estimators': [10], 
              'clf__estimator__min_samples_split': [2, 3] }

cv = GridSearchCV(pipeline,param_grid=parameters)
model=cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
prediction = cv.predict(X_test)
print(classification_report(y_test.values, prediction, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      6034
               request       0.81      0.37      0.51      1306
                 offer       0.00      0.00      0.00        43
           aid_related       0.74      0.51      0.61      3249
          medical_help       0.64      0.07      0.13       605
      medical_products       0.87      0.10      0.18       378
     search_and_rescue       0.66      0.08      0.14       250
              security       0.50      0.01      0.01       151
              military       0.40      0.03      0.06       264
                 water       0.83      0.27      0.41       500
                  food       0.84      0.31      0.45       887
               shelter       0.78      0.16      0.27       656
              clothing       0.70      0.14      0.23       102
                 money       1.00      0.04      0.08       186
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

pipeline2.get_params()



{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd17647b1e0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [19]:
parameters2 = {#'tfidf__use_idf':[True], 
             'tfidf__use_idf': [True]
              }
              

cv2 = GridSearchCV(pipeline2, param_grid=parameters2)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 45)
cv2.fit(X_train, y_train)
prediction = cv2.predict(X_test)
print(classification_report(y_test, prediction, target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      6034
               request       0.83      0.36      0.50      1306
                 offer       0.00      0.00      0.00        43
           aid_related       0.75      0.52      0.62      3249
          medical_help       0.58      0.10      0.16       605
      medical_products       0.66      0.06      0.10       378
     search_and_rescue       0.64      0.03      0.05       250
              security       0.00      0.00      0.00       151
              military       0.81      0.06      0.12       264
                 water       0.90      0.17      0.29       500
                  food       0.82      0.33      0.47       887
               shelter       0.83      0.21      0.34       656
              clothing       0.87      0.13      0.22       102
                 money       0.88      0.04      0.07       186
        missing_people       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.